This notebook has examples of how to use the CDS API to do common queries and return results that should be usable, and exportable to a Cloud Resource. A few useful links:

    The CDS Data Model: https://dataservice.datacommons.cancer.gov/#/resources
    The CDS GraphQL endpoint: https://dataservice.datacommons.cancer.gov/v1/graphql/
    The GraphiQL interface in CDS (a good place to build queries): https://dataservice.datacommons.cancer.gov/#/graphql

Import a few useful libraries. Requests for carrying out the communications and Pandas for manipulating the returns. We also use parts of IPython to format output from the dataframes to be readable.


In [36]:
import pandas as pd
import requests
from IPython.display import display, Markdown, Latex

In [37]:
cds_graphql_url = "https://dataservice.datacommons.cancer.gov/v1/graphql/"

Since there will be multiple queries, having a simple routine that runs the query and returns the answer as a JSON object will streamline the notebook

In [38]:
def runGraphQLQuery(url,query,variables):
    headers = {"content-type":"application/json"}
    try:
        if variables is None:
            results = requests.post(url = url, json={"query":query, "headers":headers})
        else:
            results = requests.post(url=url, json={"query":query, "headers":headers, "variables":variables})
        results = results.json()
    except requests.exceptions.HTTPError as exception:
        print(exception)
    return results

Let's use introspection queries to see what's avaiable in CDS

In [58]:
basic_query = """
{
  __schema{
    types{
      name
      kind
      description
    }
  }
}
"""

In [66]:
cds_fields = runGraphQLQuery(cds_graphql_url, basic_query, None)

In [67]:
field_list = cds_fields['data']['__schema']['types']

Let's take a look at this info to get an idea of what we can do with the CDS GraphQL API

In [61]:
columns = ["Name", "Kind", "Description"]
basic_df = pd.DataFrame(columns = columns)
for entry in field_list:
    if entry['description'] is None:
        description = "None"
    else:
        description = entry['description']
    basic_df.loc[len(basic_df.index)] = [entry['name'],entry['kind'],description]

In [62]:
display(Markdown(basic_df.to_markdown()))

|    | Name                        | Kind         | Description                                                                                                                                                                                           |
|---:|:----------------------------|:-------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Boolean                     | SCALAR       | Built-in Boolean                                                                                                                                                                                      |
|  1 | FileInfo                    | OBJECT       | None                                                                                                                                                                                                  |
|  2 | FileOverview                | OBJECT       | None                                                                                                                                                                                                  |
|  3 | Float                       | SCALAR       | Built-in Float                                                                                                                                                                                        |
|  4 | GS_About                    | OBJECT       | None                                                                                                                                                                                                  |
|  5 | GS_File                     | OBJECT       | None                                                                                                                                                                                                  |
|  6 | GS_Model                    | OBJECT       | None                                                                                                                                                                                                  |
|  7 | GS_Program                  | OBJECT       | None                                                                                                                                                                                                  |
|  8 | GS_Sample                   | OBJECT       | None                                                                                                                                                                                                  |
|  9 | GS_Study                    | OBJECT       | None                                                                                                                                                                                                  |
| 10 | GS_Subject                  | OBJECT       | None                                                                                                                                                                                                  |
| 11 | GlobalSearchResult          | OBJECT       | None                                                                                                                                                                                                  |
| 12 | GroupCount                  | OBJECT       | None                                                                                                                                                                                                  |
| 13 | GroupCountNeo4j             | OBJECT       | None                                                                                                                                                                                                  |
| 14 | IdsLists                    | OBJECT       | None                                                                                                                                                                                                  |
| 15 | Int                         | SCALAR       | Built-in Int                                                                                                                                                                                          |
| 16 | Mutation                    | OBJECT       | None                                                                                                                                                                                                  |
| 17 | ProgramDetail               | OBJECT       | None                                                                                                                                                                                                  |
| 18 | ProgramDetailStudy          | OBJECT       | None                                                                                                                                                                                                  |
| 19 | ProgramListEntry            | OBJECT       | None                                                                                                                                                                                                  |
| 20 | QueryType                   | OBJECT       | None                                                                                                                                                                                                  |
| 21 | RangeCount                  | OBJECT       | None                                                                                                                                                                                                  |
| 22 | RelationDirection           | ENUM         | None                                                                                                                                                                                                  |
| 23 | SampleOverview              | OBJECT       | None                                                                                                                                                                                                  |
| 24 | SearchResult                | OBJECT       | None                                                                                                                                                                                                  |
| 25 | String                      | SCALAR       | Built-in String                                                                                                                                                                                       |
| 26 | StudyDetail                 | OBJECT       | None                                                                                                                                                                                                  |
| 27 | StudyInfo                   | OBJECT       | None                                                                                                                                                                                                  |
| 28 | SubjectDetail               | OBJECT       | None                                                                                                                                                                                                  |
| 29 | SubjectOverview             | OBJECT       | None                                                                                                                                                                                                  |
| 30 | SubjectResult               | OBJECT       | None                                                                                                                                                                                                  |
| 31 | _FileInfoFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 32 | _FileInfoOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 33 | _GroupCountNeo4jFilter      | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 34 | _GroupCountNeo4jOrdering    | ENUM         | None                                                                                                                                                                                                  |
| 35 | _IdsListsFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 36 | _IdsListsOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 37 | _ProgramDetailFilter        | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 38 | _ProgramDetailOrdering      | ENUM         | None                                                                                                                                                                                                  |
| 39 | _ProgramDetailStudyFilter   | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 40 | _ProgramDetailStudyOrdering | ENUM         | None                                                                                                                                                                                                  |
| 41 | _ProgramListEntryFilter     | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 42 | _ProgramListEntryOrdering   | ENUM         | None                                                                                                                                                                                                  |
| 43 | _StudyDetailFilter          | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 44 | _StudyDetailOrdering        | ENUM         | None                                                                                                                                                                                                  |
| 45 | _StudyInfoFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 46 | _StudyInfoOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 47 | _SubjectDetailFilter        | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 48 | _SubjectDetailOrdering      | ENUM         | None                                                                                                                                                                                                  |
| 49 | __Directive                 | OBJECT       | None                                                                                                                                                                                                  |
| 50 | __DirectiveLocation         | ENUM         | An enum describing valid locations where a directive can be placed                                                                                                                                    |
| 51 | __EnumValue                 | OBJECT       | None                                                                                                                                                                                                  |
| 52 | __Field                     | OBJECT       | None                                                                                                                                                                                                  |
| 53 | __InputValue                | OBJECT       | None                                                                                                                                                                                                  |
| 54 | __Schema                    | OBJECT       | A GraphQL Introspection defines the capabilities of a GraphQL server. It exposes all available types and directives on the server, the entry points for query, mutation, and subscription operations. |
| 55 | __Type                      | OBJECT       | None                                                                                                                                                                                                  |
| 56 | __TypeKind                  | ENUM         | An enum describing what kind of type a given __Type is                                                                                                                                                |
| 57 | _diagnosisFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 58 | _diagnosisOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 59 | _fileFilter                 | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 60 | _fileOrdering               | ENUM         | None                                                                                                                                                                                                  |
| 61 | _genomic_infoFilter         | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 62 | _genomic_infoOrdering       | ENUM         | None                                                                                                                                                                                                  |
| 63 | _participantFilter          | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 64 | _participantOrdering        | ENUM         | None                                                                                                                                                                                                  |
| 65 | _programFilter              | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 66 | _programOrdering            | ENUM         | None                                                                                                                                                                                                  |
| 67 | _sampleFilter               | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 68 | _sampleOrdering             | ENUM         | None                                                                                                                                                                                                  |
| 69 | _specimenFilter             | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 70 | _specimenOrdering           | ENUM         | None                                                                                                                                                                                                  |
| 71 | _studyFilter                | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 72 | _studyOrdering              | ENUM         | None                                                                                                                                                                                                  |
| 73 | _treatmentFilter            | INPUT_OBJECT | None                                                                                                                                                                                                  |
| 74 | _treatmentOrdering          | ENUM         | None                                                                                                                                                                                                  |
| 75 | diagnosis                   | OBJECT       | None                                                                                                                                                                                                  |
| 76 | file                        | OBJECT       | None                                                                                                                                                                                                  |
| 77 | genomic_info                | OBJECT       | None                                                                                                                                                                                                  |
| 78 | participant                 | OBJECT       | None                                                                                                                                                                                                  |
| 79 | program                     | OBJECT       | None                                                                                                                                                                                                  |
| 80 | sample                      | OBJECT       | None                                                                                                                                                                                                  |
| 81 | specimen                    | OBJECT       | None                                                                                                                                                                                                  |
| 82 | study                       | OBJECT       | None                                                                                                                                                                                                  |
| 83 | treatment                   | OBJECT       | None                                                                                                                                                                                                  |

There's quite a lot of ways to query CDS, the one's to pay attention to are those with the OBJECT label.  They can also take an argument (we'll get into that later) to narrow down what gets returned.

Another introspection query that's worth looking through involved the queryType.

In [47]:
query_type_query = """
{
  __schema{
    queryType{
      fields{
        name
        description
      }
    }
  }
}
"""

In [48]:
query_type_results = runGraphQLQuery(cds_graphql_url, query_type_query, None)

In [54]:
query_type_list = query_type_results['data']['__schema']['queryType']['fields']

In [55]:
columns = ["Name", "Description"]
query_type_df = pd.DataFrame(columns=columns)
for entry in query_type_list:
    #print(entry)
    if entry['description'] is None:
        description = "None"
    else:
        description = entry['description']
    query_type_df.loc[len(query_type_df.index)] = [entry['name'],description]

In [56]:
display(Markdown(query_type_df.to_markdown()))

|    | Name                 | Description                                      |
|---:|:---------------------|:-------------------------------------------------|
|  0 | diagnosis            | None                                             |
|  1 | file                 | None                                             |
|  2 | genomic_info         | None                                             |
|  3 | participant          | None                                             |
|  4 | program              | None                                             |
|  5 | sample               | None                                             |
|  6 | specimen             | None                                             |
|  7 | study                | None                                             |
|  8 | treatment            | None                                             |
|  9 | schemaVersion        | None                                             |
| 10 | numberOfStudies      | None                                             |
| 11 | numberOfSubjects     | None                                             |
| 12 | numberOfSamples      | None                                             |
| 13 | numberOfFiles        | None                                             |
| 14 | numberOfDiseaseSites | None                                             |
| 15 | idsLists             | Get lists of all subject ids, used by Local Find |
| 16 | studyDetail          | Study Detail Page                                |
| 17 | subjectDetail        | Subject Detail Page                              |
| 18 | samplesForSubjectId  | None                                             |
| 19 | studyList            | None                                             |
| 20 | programDetail        | None                                             |
| 21 | programList          | None                                             |
| 22 | fileInfo             | None                                             |
| 23 | studyInfo            | None                                             |
| 24 | programDetailStudy   | None                                             |
| 25 | programListEntry     | None                                             |
| 26 | groupCountNeo4j      | None                                             |
| 27 | searchSubjects       | None                                             |
| 28 | subjectOverview      | None                                             |
| 29 | sampleOverview       | None                                             |
| 30 | fileOverview         | None                                             |
| 31 | fileIDsFromList      | None                                             |
| 32 | findSubjectIdsInList | None                                             |
| 33 | globalSearch         | None                                             |

These are the fields that generally contain data and could be of use when conducting a search for data

Lastly, it's worth understanding what is contained in the OBJECT fields from the first query.  To do this, we'll build a query that has a variable built in so we can run it for each field

In [68]:
field_query = """
query cdsFields ($fieldname:String!) {
  __type(name: $fieldname){
    fields{
      name
    }
  }
}
"""

Now loop throught all the fields and build a structure that contains each enty has a listing of it's fields

In [78]:
final_data = {}
for entry in field_list:
    templist = []
    variables = {"fieldname": entry['name']}
    results = runGraphQLQuery(cds_graphql_url, field_query, variables)
    if results['data']['__type']['fields'] is not None:
            for item in results['data']['__type']['fields']:
                for key, name in item.items():
                    templist.append(name)
    final_data[entry['name']] = templist

In [79]:
columns = ['Query Fields', 'Associated Fields']
query_df = pd.DataFrame(columns = columns)
for key, value in final_data.items():
    query_df.loc[len(query_df.index)] = [key, value]


In [80]:
display(Markdown(query_df.to_markdown()))

|    | Query Fields                | Associated Fields                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|---:|:----------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Boolean                     | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  1 | FileInfo                    | ['subject_id', 'file_name', 'file_type', 'association', 'file_description', 'file_size', 'file_id', 'md5sum']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|  2 | FileOverview                | ['study_acronym', 'accesses', 'phs_accession', 'subject_id', 'sample_id', 'experimental_strategy', 'gender', 'analyte_type', 'is_tumor', 'file_name', 'file_type', 'file_size', 'file_id', 'md5sum']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|  3 | Float                       | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  4 | GS_About                    | ['page', 'title', 'type', 'text']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  5 | GS_File                     | ['type', 'subject_id', 'sample_id', 'file_id', 'file_name', 'file_type', 'accesses', 'acl', 'experimental_strategies', 'instrument_models', 'library_layouts', 'library_selections', 'library_sources', 'library_strategies', 'platforms', 'reference_genome_assemblies', 'sites']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  6 | GS_Model                    | ['type', 'node_name', 'property_name', 'property_description', 'property_required', 'property_type', 'value', 'highlight']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  7 | GS_Program                  | ['program_name', 'program_short_description', 'program_full_description', 'program_external_url', 'program_sort_order', 'type']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  8 | GS_Sample                   | ['type', 'sample_id', 'is_tumor', 'analyte_type']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  9 | GS_Study                    | ['type', 'phs_accession', 'study_data_types', 'study_name', 'study_code']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 10 | GS_Subject                  | ['type', 'study', 'subject_id', 'site', 'gender']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 11 | GlobalSearchResult          | ['study_count', 'studies', 'subject_count', 'subjects', 'sample_count', 'samples', 'file_count', 'files', 'program_count', 'programs', 'about_count', 'about_page', 'model_count', 'model']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 12 | GroupCount                  | ['group', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 13 | GroupCountNeo4j             | ['group', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 14 | IdsLists                    | ['subjectIds']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 15 | Int                         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 16 | Mutation                    | ['creatediagnosis', 'createfile', 'creategenomic_info', 'createparticipant', 'createprogram', 'createsample', 'createspecimen', 'createstudy', 'createtreatment', 'createFileInfo', 'createSubjectDetail', 'createStudyDetail', 'createStudyInfo', 'createIdsLists', 'createProgramDetail', 'createProgramDetailStudy', 'createProgramListEntry', 'createGroupCountNeo4j']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 17 | ProgramDetail               | ['program', 'program_name', 'program_url', 'program_short_description', 'num_studies', 'num_participants', 'num_files', 'num_samples', 'num_disease_sites', 'study_participants', 'studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 18 | ProgramDetailStudy          | ['accession', 'study_access', 'study_name', 'study_description', 'short_description', 'num_participants', 'num_samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 19 | ProgramListEntry            | ['acronym', 'name', 'website', 'num_studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 20 | QueryType                   | ['diagnosis', 'file', 'genomic_info', 'participant', 'program', 'sample', 'specimen', 'study', 'treatment', 'schemaVersion', 'numberOfStudies', 'numberOfSubjects', 'numberOfSamples', 'numberOfFiles', 'numberOfDiseaseSites', 'idsLists', 'studyDetail', 'subjectDetail', 'samplesForSubjectId', 'studyList', 'programDetail', 'programList', 'fileInfo', 'studyInfo', 'programDetailStudy', 'programListEntry', 'groupCountNeo4j', 'searchSubjects', 'subjectOverview', 'sampleOverview', 'fileOverview', 'fileIDsFromList', 'findSubjectIdsInList', 'globalSearch']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 21 | RangeCount                  | ['lowerBound', 'upperBound', 'subjects']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 22 | RelationDirection           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 23 | SampleOverview              | ['study_acronym', 'phs_accession', 'subject_id', 'sample_id', 'is_tumor', 'analyte_type', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 24 | SearchResult                | ['numberOfStudies', 'numberOfSubjects', 'numberOfSamples', 'numberOfFiles', 'numberOfDiseaseSites', 'subjectCountByStudy', 'subjectCountByExperimentalStrategy', 'subjectCountByAccess', 'subjectCountByGender', 'subjectCountByIsTumor', 'subjectCountByAnalyteType', 'subjectCountByFileType', 'subjectCountByDiseaseSite', 'subjectCountByLibraryStrategy', 'subjectCountByLibrarySource', 'subjectCountByLibrarySelection', 'subjectCountByLibraryLayout', 'subjectCountByPlatform', 'subjectCountByInstrumentModel', 'subjectCountByReferenceGenomeAssembly', 'subjectCountByPrimaryDiagnosis', 'subjectCountByPhsAccession', 'subjectCountByStudyDataType', 'subjectCountByAcl', 'filterSubjectCountByStudy', 'filterSubjectCountByExperimentalStrategy', 'filterSubjectCountByAccess', 'filterSubjectCountByGender', 'filterSubjectCountByIsTumor', 'filterSubjectCountByFileType', 'filterSubjectCountByAnalyteType', 'filterSubjectCountByDiseaseSite', 'filterSubjectCountByLibraryStrategy', 'filterSubjectCountByLibrarySource', 'filterSubjectCountByLibrarySelection', 'filterSubjectCountByLibraryLayout', 'filterSubjectCountByPlatform', 'filterSubjectCountByInstrumentModel', 'filterSubjectCountByReferenceGenomeAssembly', 'filterSubjectCountByPrimaryDiagnosis', 'filterSubjectCountByPhsAccession', 'filterSubjectCountByStudyDataType', 'filterSubjectCountByAcl', 'filterSubjectCountByNumberOfStudyParticipants', 'filterSubjectCountByNumberOfStudySamples'] |
| 25 | String                      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 26 | StudyDetail                 | ['study_name', 'phs_accession', 'study_acronym', 'study_description', 'data_types', 'study_external_url', 'numberOfSubjects', 'numberOfSamples', 'numberOfDiseaseSites', 'numberOfFiles']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 27 | StudyInfo                   | ['study_name', 'phs_accession', 'data_type', 'numberOfSubjects', 'numberOfFiles', 'study_access']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 28 | SubjectDetail               | ['subject_id', 'study_acronym', 'study_name', 'gender', 'site', 'files', 'samples', 'numberOfSamples', 'numberOfDiseaseSites', 'numberOfFiles']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 29 | SubjectOverview             | ['subject_id', 'study_acronym', 'phs_accession', 'gender', 'site', 'samples', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| 30 | SubjectResult               | ['subject_id', 'phs_accession']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 31 | _FileInfoFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 32 | _FileInfoOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 33 | _GroupCountNeo4jFilter      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 34 | _GroupCountNeo4jOrdering    | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 35 | _IdsListsFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 36 | _IdsListsOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 37 | _ProgramDetailFilter        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 38 | _ProgramDetailOrdering      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 39 | _ProgramDetailStudyFilter   | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 40 | _ProgramDetailStudyOrdering | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 41 | _ProgramListEntryFilter     | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 42 | _ProgramListEntryOrdering   | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 43 | _StudyDetailFilter          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 44 | _StudyDetailOrdering        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 45 | _StudyInfoFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 46 | _StudyInfoOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 47 | _SubjectDetailFilter        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 48 | _SubjectDetailOrdering      | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 49 | __Directive                 | ['name', 'description', 'isRepeatable', 'locations', 'args']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 50 | __DirectiveLocation         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 51 | __EnumValue                 | ['name', 'description', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 52 | __Field                     | ['name', 'description', 'args', 'type', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 53 | __InputValue                | ['name', 'description', 'type', 'defaultValue', 'isDeprecated', 'deprecationReason']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 54 | __Schema                    | ['description', 'types', 'queryType', 'mutationType', 'directives', 'subscriptionType']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 55 | __Type                      | ['kind', 'name', 'description', 'fields', 'interfaces', 'possibleTypes', 'enumValues', 'inputFields', 'ofType', 'specifiedByUrl']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 56 | __TypeKind                  | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 57 | _diagnosisFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 58 | _diagnosisOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 59 | _fileFilter                 | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 60 | _fileOrdering               | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 61 | _genomic_infoFilter         | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 62 | _genomic_infoOrdering       | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 63 | _participantFilter          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 64 | _participantOrdering        | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 65 | _programFilter              | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 66 | _programOrdering            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 67 | _sampleFilter               | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 68 | _sampleOrdering             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 69 | _specimenFilter             | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 70 | _specimenOrdering           | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 71 | _studyFilter                | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 72 | _studyOrdering              | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 73 | _treatmentFilter            | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 74 | _treatmentOrdering          | []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 75 | diagnosis                   | ['diagnosis_id', 'disease_type', 'vital_status', 'primary_diagnosis', 'primary_site', 'age_at_diagnosis', 'tumor_grade', 'tumor_stage_clinical_m', 'tumor_stage_clinical_n', 'tumor_stage_clinical_t', 'morphology', 'incidence_type', 'progression_or_recurrence', 'days_to_recurrence', 'days_to_last_followup', 'last_known_disease_status', 'days_to_last_known_status', 'participant']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 76 | file                        | ['file_id', 'file_name', 'file_type', 'file_description', 'file_size', 'md5sum', 'file_url_in_cds', 'experimental_strategy_and_data_subtypes', 'study', 'samples', 'genomic_info']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 77 | genomic_info                | ['library_id', 'bases', 'number_of_reads', 'avg_read_length', 'coverage', 'reference_genome_assembly', 'custom_assembly_fasta_file_for_alignment', 'design_description', 'library_strategy', 'library_layout', 'library_source', 'library_selection', 'platform', 'instrument_model', 'sequence_alignment_software', 'files', 'samples', 'file']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 78 | participant                 | ['participant_id', 'race', 'gender', 'ethnicity', 'dbGaP_subject_id', 'study', 'diagnoses', 'specimens', 'samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 79 | program                     | ['program_name', 'program_acronym', 'program_short_description', 'program_full_description', 'program_external_url', 'program_sort_order', 'studies']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 80 | sample                      | ['sample_id', 'sample_type', 'sample_tumor_status', 'sample_anatomic_site', 'sample_age_at_collection', 'derived_from_specimen', 'biosample_accession', 'participant', 'specimen', 'files', 'genomic_info']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 81 | specimen                    | ['specimen_id', 'participant', 'samples']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 82 | study                       | ['study_name', 'study_acronym', 'study_description', 'short_description', 'study_external_url', 'primary_investigator_name', 'primary_investigator_email', 'co_investigator_name', 'co_investigator_email', 'phs_accession', 'bioproject_accession', 'index_date', 'cds_requestor', 'funding_agency', 'funding_source_program_name', 'grant_id', 'clinical_trial_system', 'clinical_trial_identifier', 'clinical_trial_arm', 'organism_species', 'adult_or_childhood_study', 'data_types', 'file_types', 'data_access_level', 'cds_primary_bucket', 'cds_secondary_bucket', 'cds_tertiary_bucket', 'number_of_participants', 'number_of_samples', 'study_data_types', 'file_types_and_format', 'size_of_data_being_uploaded', 'size_of_data_being_uploaded_unit', 'size_of_data_being_uploaded_original', 'size_of_data_being_uploaded_original_unit', 'acl', 'study_access', 'program', 'participants', 'files']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 83 | treatment                   | ['treatment_id', 'treatment_type', 'treatment_outcome', 'days_to_treatment', 'therapeutic_agents']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |

At this point, we've used GraphQL introspection to get a handle on what's in the database and how we might access it.  In our next notebooke, we'll build on this knowledge to search for files that we can use in analyses.

CDS has multiple studies, so as a first step, it's worth taking a look to see what content is avaialble.  This first query simply lists out the different studies that have data (This notebook assumes that you've done some basic introspection queries to get familiar wtih the content of the CDS database).

In [14]:
study_query = """
{
  study{
    study_name
    study_description
    study_acronym
    study_access
    phs_accession
  }
}
"""

This query will bring back several useful pieces of information that should help decide what studies are worth further examination.  You can also run this query in the GraphiQL interface in the CDS portal (https://dataservice.datacommons.cancer.gov/#/graphql) to see what the JSON object looks like.

In [16]:
study_result = runGraphQLQuery(cds_graphql_url, study_query, None)

Let's put the information in a dataframe so we can easily make it more readable than a JSON object

In [17]:
df_columns = ['Study Name', 'Acronym', 'Access', 'Accession', 'Description']
study_df = pd.DataFrame(columns = df_columns)
for study in study_result['data']['study']:
    study_df.loc[len(study_df.index)] = [study['study_name'],study['study_acronym'],study['study_access'], study['phs_accession'],study['study_description']]

In [18]:
display(Markdown(study_df.to_markdown()))

|    | Study Name                                                                                                                                    | Acronym              | Access     | Accession   | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|---:|:----------------------------------------------------------------------------------------------------------------------------------------------|:---------------------|:-----------|:------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Clinical Trial Specimen Molecular Characterization (CTSMC)                                                                                    | CTSMC                | Controlled | phs002790   | The National Cancer Institute's (NCI) Childhood Cancer Data Initiative (CCDI) focuses on the critical need to collect, analyze, and share data to address the burden of cancer in children, adolescents, and young adults (AYAs). The Molecular Characterization Initiative (MCI) will further the CCDI's goals by providing access to better diagnostic tests for pediatric and AYA patients. The molecular characterizations of solid tumors, soft tissue sarcomas, and rare diseases are performed in a CLIA-certified setting as results may be used to screen for and/or confirm clinical trial eligibility, direct treatment, or otherwise contribute to the conduct of the trial. The following molecular characterization assays were performed:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|  1 | UCSF Database for the Advancement of JMML - Integration of Metadata with ""Omic"" Data                                                        | JMML                 | Controlled | phs002504   | Juvenile myelomonocytic leukemia (JMML) is a rare and frequently fatal myeloproliferative/myelodysplastic disorder of early childhood with an estimated incidence of 1.2 cases per million. It is associated with a spectrum of diverse outcomes ranging from spontaneous resolution in rare patients to transformation to acute myeloid leukemia in others. The overwhelming majority of JMML patients (~95%) will harbor mutations in canonical Ras pathway genes, including NF1, NRAS, KRAS, PTPN11, and CBL. As Ras proteins are mutated in more than 30% of human cancers, the information gleaned from the study of JMML has provided insights into Ras signaling in cancer as well as a group of congenital diseases with tumor predispositions known as the “Rasopathies”. While oncogenic Ras is one of the most common mutations in human cancer, it remains one of the most vexing targets for efficacious therapy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  2 | Human Tumor Atlas Network (HTAN) primary sequencing data                                                                                      | HTAN                 | Controlled | phs002371   | An NCI-funded Cancer Moonshot initiative to construct 3-dimensional atlases of the dynamic cellular, morphological, and molecular features of human cancers as they evolve from precancerous lesions to advanced disease.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|  3 | CIDR: Discovery, Biology, and Risk of Inherited Variants in Glioma sample                                                                     |                      | Controlled | phs002250   | This is a gliogene brain tumor family study. This study includes glioma cases with a family history of glioma in a first, second, or third degree relative.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Study Weblinks:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Gliogene                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Family/Twin/Trios                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Family                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             | Whole Genome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | dbGaP estimated ancestry using GRAF-pop                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 151                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|  4 | Childhood Cancer Data Initiative (CCDI): Free the Data: Open Sharing of Comprehensive Genomic Childhood Cancer Datasets (Kansas)              | CCDI-KUMC            | Controlled | phs002529   | This study provides paired tumor normal genomic sequencing data from approximately 200 children with cancer, including both solid tumors and leukemias, done by the Children’s Mercy Research Institute (CMRI) and University of Kansas Cancer Center (KUCC).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | These data include whole genome sequencing (generally, ~20x), whole exome sequencing (generally, ~300x), bulk RNA sequencing (generally, ~80 million reads), and single-cell RNA and ATAC sequencing (>50,000 reads/cell). Additional phenotypic, pathologic, and genetic data, gathered clinically for these samples, are also provided.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             |     Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Clinical Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Clinical Genetic Testing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Exome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | Full Transcriptome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Individual-Level Genomic Data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Mixed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                                                               |                      |            |             | Probands                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Repository                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                                                               |                      |            |             | RNA Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                                                               |                      |            |             | Single Cell Analysis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                                                               |                      |            |             | Transcriptome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | TumorTumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Whole Genome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 193                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|  5 | University of Texas PDX Development and Trial Center Grant                                                                                    |                      | Controlled | phs001980   | The goal for the University Texas PDX Development and Trial Center (UTPDTC) is to optimize personalized biomarker-based cancer therapy and identify effective targeted drugs based on the molecular characteristics of each tumor. Our short-term goals are to establish a biobank of clinically, and molecularly-annotated Patient-Derived Xenografts (PDXs) and to use PDXs as a platform for preclinical drug development and biomarker discovery. The primary goal for UTPDTC investigators will be to develop PDX trial strategies for preclinical testing of single agents and drug combinations. These models will allow the determination of the optimal treatments (single drugs or combinations) that should be tested in clinical trials in increasingly individualized, molecularly defined subsets of tumors. The goal of the Patient-Derived Xenograft Core is to provide high-quality clinically relevant and molecularly annotated PDX models for the research projects proposed in the University of Texas PDX Development and Trial Center (UTPDTC) grant application and to the research activity of the NCI PDX Development and Trial Centers Research Network (PDXNet) by leveraging PDX resources at our institutions and developing new PDX models from human cancer specimens using rigorous quality standards so that the models can be used to guide clinical trial development. The PDX models developed and/or characterized by the PDX Core will be available to other cancer researchers through the PDXNet and NCI Patient-Derived Models Repository (PDMR).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             |     Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                                                               |                      |            |             |         Prospective Longitudinal Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             |     Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             |         Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             |     Total number of consented subjects: 36                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  6 | Clonal evolution during metastatic spread in high-rish neuroblastoma                                                                          | CEMSHRN              | Controlled | phs003111   | The goal of this study is to deliver a detailed characterization of the patterns of disease dissemination at diagnosis, during progression and in response to therapy in high-risk neuroblastoma. Longitudinal and spatially distinct tumors were collected from patients. Clinical data includes diagnosis and treatment information. Biospecimen data includes whole genome sequencing (WGS) and whole transcriptome sequencing (WTS).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Longitudinal Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             | Transcriptome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Whole Genome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 129                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|  7 | LCCC 1108: Development of a Tumor Molecular Analyses Program and Its Use to Support Treatment Decisions (UNCseqTM)                            | LCCC 1108 (UNCseqTM) | Controlled | phs001713   | The primary objective of this specimen correlative study was two-fold: to provide a mechanism for the association of known molecular alterations with clinical outcomes, and to provide rapid genetic profiling of alterations with known clinical utility using tumor and germline specimens to support treatment decisions.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|  8 | Washington University PDX Development and Trial Center                                                                                        |                      | Controlled | phs002305   | This data set is a collection of patient specimens from common and rare cancers that have been developed into patient-derived xenograft (PDX) models at the Washington University PDX Development and Trial Center (WU-PDTC). The goal of the PDTC is to develop and characterize PDX models, gaining insight into tumor biology, and validating biomarkers across all major tumor types. Pre-clinical experiments in these PDX models will be used to advance our ability to predict clinical responses to new molecularly targeted agents under development.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Tumor                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                                                               |                      |            |             | Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 127                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|  9 | CPTAC Pan-Cancer                                                                                                                              | CPTAC                | Controlled | phs001287   | Recently, significant progress has been made in characterizing and sequencing the genomic alterations in statistically robust numbers of samples from several types of cancer. For example, The Cancer Genome Atlas (TCGA), International Cancer Genome Consortium (ICGC) and other similar efforts are identifying genomic alterations associated with specific cancers (e.g., copy number aberrations, rearrangements, point mutations, epigenomic changes, etc.) The availability of these multi-dimensional data to the scientific community sets the stage for the development of new molecularly targeted cancer interventions. Understanding the comprehensive functional changes in cancer proteomes arising from genomic alterations and other factors is the next logical step in the development of high-value candidate protein biomarkers. Hence, proteomics can greatly advance the understanding of molecular mechanisms of disease pathology via the analysis of changes in protein expression, their modifications and variations, as well as protein=protein interaction, signaling pathways and networks responsible for cellular functions such as apoptosis and oncogenesis. Realizing this great potential, the NCI launched the third phase of the CPTC initiative in September 2016. As the Clinical Proteomic Tumor Analysis Consortium, CPTAC continues to define cancer proteomes on genomically-characterized biospecimens. The purpose of this integrative approach was to provide the broad scientific community with knowledge that links genotype to proteotype and ultimately phenotype. In this third phase of CPTAC, the program aims to expand on CPTAC II and genomically and proteomically characterize over 2000 samples from 10 cancer types (Lung Adenocarcinoma, Pancreatic Ductal Adenocarcinoma, Glioblastoma Multiforme, Acute Myeloid Leukemia, Clear cell renal Carcinoma, Head and Neck Squamous Cell Carcinoma, Cutaneous Melanoma, Sarcoma, Lung Squamous Cell Carcinoma, Uterine Corpus Endometrial Carcinoma) .Germline DNA is obtained from blood and Normal control samples for proteomics varied by organ site. All cancer samples were derived from primary and untreated tumor. |
| 10 | NCI CCSG CCDI Supplement Additional Genomic Submission                                                                                        |                      | Controlled | phs002599   | The implementation of targeted therapies for acute myeloid leukaemia (AML) has been challenging because of the complex mutational patterns within and across patients as well as a dearth of pharmacologic agents for most mutational events. This supplement data from the Beat AML program provides Adolescent and Young Adult (AYA) and pediatric tumor specimens not previously released. These data include exome and RNA level characterization as well as analyses of ex vivo drug sensitivity. Collectively, this data can be leveraged to address clinical, genomic, transcriptomic and functional analyses of the biology of leukemia in pediatric and AYA populations.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Exome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | RNA Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 104                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 11 | CIDR: The Genetic Basis of Aggressive Prostate Cancer: The Role of Rare Variation                                                             |                      | Controlled | phs001524   | In this study, we will apply a multi-staged approach to reveal genes harboring rare variants that are associated with aggressive PCa. Whole-exome sequencing (Aim 1a) of 2,774 aggressive cases and 2,776 non-aggressive cases of European ancestry will be conducted followed by rare variant analysis of single sites and gene burden testing to identify novel susceptibility loci/genes for aggressive disease. We will validate the most significantly associated genes (~500) through targeted sequencing in an additional 7,500 aggressive and 7,500 non-aggressive cases (Aim 1b). Next, we will investigate the clinical predictive utility of the genes/variants identified in 2,300 cases in the STHM3 trial who are undergoing biopsy based on PSA and genetic risk score stratification (Aim 2). Through this tiered approach we expect to significantly advance knowledge of aggressive PCa etiology and health disparities as well as guide the development of early detection and prognostic strategies for the subset of men who are most susceptible to this fatal form of disease. In this case-case study of aggressive vs non aggressive prostate cancer, aggressive cases are defined as prostate cancer as cause of death, (T4 disease or T3 disease) and Gleason 8+. Non-aggressive cases are men with T1/2 disease and Gleason <=6.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | CONTRIBUTING SITES                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | CAPS, PROCAP, STHM1, STHM2: Swedish Cancer Society (CAN 2016/818), Swedish Research Council (2014/2269)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | MEC: Funding provided by the National Cancer Institute: Understanding Ethnic Differences in Cancer, 2U01CA164973 and The Genetic Basis of Aggressive Prostate Cancer, The Role of Rare Variation, 5R01CA196931-02.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | CPSII: The authors express sincere appreciation to all Cancer Prevention Study II participants and to each member of the study and biospecimen management group. The American Cancer Society funds the creation, maintenance, and updating of the Cancer Prevention Study-II cohort.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | MCCS/APCS/PCFS: The Melbourne Collaborative Cohort Study (MCCS) recruitment was funded by VicHealth and Cancer Council Victoria and further supported by Australian National Health and Medical Research Council (NHMRC) grants 209057 and 396414. The Aggressive Prostate Cancer Case-Control Study (APCS) was funded by NHMRC grant 623204. The Prostate Cancer Family Study (PCFS) was fully funded by Cancer Council Victoria. Cancer Council Victoria funds the continuing maintenance and updating of the MCCS, APCS and PCFS. Cases and their vital status are ascertained and followed up through the Victorian Cancer Registry and the Australian Institute of Health and Welfare, including the National Death Index and the Australian Cancer Database.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | PLCO: The Prostate Lung Colorectal Ovarian Cancer Screening Trial (PLCO) was supported by the Intramural Research Program of the Division of Cancer Epidemiology and Genetics and by contracts from the Division of Cancer Prevention, National Cancer Institute, US National Institutes of Health, Department of Health and Human Services.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | ATBC: The ATBC Study is supported by the Intramural Research Program of the U.S. National Cancer Institute, National Institutes of Health, and by U.S. Public Health Service contract HHSN261201500005C from the National Cancer Institute, Department of Health and Human Services.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | ICR: This work was supported by the NIH R01 grant 5R01CA196931-02.The samples from the UK were from UKGPCS and PrompT. The UKGPCS study was supported by Cancer Research UK (grant numbers C5047/A7357, C1287/A10118, C1287/A5260, C5047/A3354, C5047/A10692, C16913/A6135 and C16913/A6835). We would like to acknowledge the NCRN nurses and Consultants for their work in the UKGPCS study. We thank all the patients who took part in this study. We also acknowledge The Institute of Cancer Research, The National Cancer Research Network UK, The National Cancer Research Institute (NCRI) UK for their ongoing support. We are grateful for support of NIHR funding to the NIHR Biomedical Research Centre at The Institute of Cancer Research and The Royal Marsden NHS Foundation Trust.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | CIDR grant X01HG008336                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | dbGaP estimated ancestry using GRAF-pop                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 5563                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 12 | Feasibility and Clinical Utility of Whole Genome Profiling in Pediatric and Young Adult Cancers                                               |                      | Controlled | phs002620   | Comprehensive Whole Genome Sequencing and Whole Transcriptome (RNASeq) analyses from 114 pediatric patients at Memorial Sloan Kettering. This dataset was used as a prospective clinical utility and feasibility study. By using our Isabl infrastructure for precision medicine, we developed a 2-week end-to-end pipeline to analyze cfDNA, WGS and WTS, which we refer to as cWGTS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 13 | CIDR: The Role of Rare Coding Variation in Prostate Cancer in Men of African Ancestry - RESPOND Project 2                                     |                      | Controlled | phs002637   | "In RESPOND Project 2, we seek to identify rare genetic factors that are associated with prostate cancer (PCa) risk and aggressiveness in men of African ancestry (AA). We will conduct exome sequencing of 15,000 prostate cancer cases and 5,000 controls from the RESPOND cohort and the African Ancestry Prostate Cancer Consortium (AAPC) with cases selected based on risk categories: high-risk (stage T3/T4 or Gleason 8+ or PSA>20 ng/ml), intermediate-risk (stage T2b/T2c or Gleason 7 or PSA 10-20 ng/ml) and low-risk disease (stage T1/T2a and Gleason ≤ 6 and PSA<10 ng/ml). We expect the findings from this Project to significantly advance knowledge of susceptibility to aggressive PCa and racial/ethnic disparities in PCa risk, and to guide the development of future preventive, early detection and prognostic measures for AA men. The first phase of the study will include exome sequence data for approximately 7,500 cases and 5,000 controls from AAPC. The second phase of the study will include exome data for 5,000 cases from RESPOND and AAPC.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | ACKNOWLEDGMENTS and CONTRIBUTING SITES                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Multiethnic Cohort (MEC): The MEC and the genotyping in this study were supported by National Institutes of Health (NIH) grants CA63464, CA54281, CA1326792, CA148085, and HG004726. Cancer incidence data for the MEC and Los Angeles Study of Aggressive Prostate Cancer (LAAPC) studies have been collected by the Los Angeles Cancer Surveillance Program of the University of Southern California with federal funds from the National Cancer Institute (NCI)/NIH/Department of Health and Human Services (DHHS) under Contract No. N01-PC-35139, and the California Department of Health Services as part of the state-wide cancer reporting program mandated by California Health and Safety Code Section 103885, and grant 1U58DP000807-3 from the Centers for Disease Control and Prevention (CDC).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Ghana Prostate Study (GPS): The Ghana Prostate Study was funded by the Intramural Program of the NCI/NIH/DHHS under Contract No. HHSN261200800001E.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Men of African Descent and Carcinoma of the Prostate (MADCaP): We thank all MADCaP study participants. This work is a product of the MADCaP network. This work was supported by NCI/NIH grant U01CA184374 to Timothy Rebbeck and National Institute of General Medical Sciences (NIGMS) MIRA grant R35GM133727 to Joseph Lachance. Additional funding includes a seed grant from the Integrated Cancer Research Center at Georgia Institute of Technology.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | UGANDA: The UGANDA study was supported by NIH grant R01CA165862.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Southern Community Cohort (SCCS) is funded by NIH grant CA092447. SCCS sample preparation was conducted at the Epidemiology Biospecimen Core Lab that is supported in part by the Vanderbilt Ingram Cancer Center (CA68485). Data on SCCS cancer cases were provided by Alabama Statewide Cancer Registry, Kentucky Cancer Registry, Office of Cancer Surveillance at Tennessee Department of Health, Florida Cancer Data System, Central Cancer Registry at North Carolina Division of Public Health, Georgia Comprehensive Cancer Registry, Louisiana Tumor Registry, Mississippi Cancer Registry, South Carolina Central Cancer Registry, Virginia Cancer Registry at Virginia Department of Health, and Cancer Registry at Arkansas Department of Health. The Arkansas Central Cancer Registry is fully funded by a grant from National Program of Cancer Registries (NPCR)/Centers for Disease Control and Prevention (CDC). Data on SCCS cancer cases from Mississippi were collected by the Mississippi Cancer Registry which participates in the NPCR/CDC. The contents of this publication are solely the responsibility of the authors and do not necessarily represent the official views of the CDC or the Mississippi Cancer Registry.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Karuprostate: The Karuprostate study was supported by the French National Health Directorate and by the Association pour la Recherche sur les Tumeurs de la Prostate. Séverine Ferdinand, Marc Romana.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | The North Carolina - Louisiana Prostate Cancer Project (PCaP) is carried out as a collaborative study supported by the Department of Defense contract DAMD 17-03-2-0052. The authors thank the staff, advisory committees and research subjects participating in the PCaP study for their important contributions. We would also like to acknowledge the UNC BioSpecimen Facility and the LSUHSC Pathology Lab for our DNA extractions, blood processing, storage and sample disbursement (https://genome.unc.edu/bsp).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Gene-Environment Interaction in Prostate Study (GECAP) was supported by NIH grant ES011126.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | King County Prostate Cancer Study (KCPCS) was supported by NIH grants CA056678, CA082664, and CA092579, with additional support from the Fred Hutchinson Cancer Research Center. We thank the participants in these studies, and Ms. Suzanne Kolb for help with study management.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | The Los Angeles Study of Aggressive Prostate Cancer (LAAPC) was funded by grant 99-00524V-10258 from the Cancer Research Fund, under Interagency Agreement #97-12013 (University of California contract #98-00924V) with the Department of Health Services Cancer Research Program. Cancer incidence data for the MEC and LAAPC studies have been collected by the Los Angeles Cancer Surveillance Program of the University of Southern California with Federal funds from the NCI/NIH/DHHS under Contract No. N01-PC-35139, and the California Department of Health Services as part of the state-wide cancer reporting program mandated by California Health and Safety Code Section 103885, and grant 1U58DP000807-3 from the CDC.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Prostate Cancer Studies at MD Anderson (MDA) was supported by grants CA68578, ES007784, DAMD W81XWH-07-1-0645, and CA140388.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Case-Control                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case-Control                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             | Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             | Exome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 12049"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 14 | Childhood Cancer Data Initiative (CCDI): Integration of genomic and clinical data from unique rare cancer datasets to facilitate data sharing | CBTN                 | Controlled | phs002517   | This sequencing cohort defines the largest, clinically annotated pediatric brain tumor cohort study to date and seeks to define the intersection of germline and somatic underpinnings of pediatric brain tumors across a shared developmental context of cancer and structural birth defects.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Study Weblinks:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Childrens Brain Tumor Network                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Pacific Pediatric Neuro-Oncology Consortium                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | CHOP Division of Genomic Diagnostics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                                                               |                      |            |             | Kids First Data Resource Portal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | PedCBio Portal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Clinical Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Clinical Genetic Testing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Clinical Trial                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Copy Number Variation (CNV)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Exome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | Genotype/Expression                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             | Individual-Level Genomic Data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Longitudinal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                                                               |                      |            |             | Longitudinal Cohort                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                                                               |                      |            |             | Multicenter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Observational                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Parent-Offspring                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | Parent-Offspring Trios                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             | Probands                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Prospective                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | RNA Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                                                               |                      |            |             | Tissue Expression                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                                                               |                      |            |             | Tumor                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                                                               |                      |            |             | Tumor vs. Matched-Normal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Whole Genome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 1031                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 15 | Wistar PDX Development and Trial Center SPORE in Skin Cancer                                                                                  |                      | Controlled | phs002432   | Overall the outcomes of patients with metastatic melanoma have improved dramatically over the last decade due to an improved understanding of the molecular drivers of this disease. In particular, multiple targeted therapy regimens have been approved for patients with a BRAFV600E/K mutation, which are present in ~50% of cutaneous melanomas. These treatments achieve clinical responses in ~80% of patients with a BRAFV600E/K mutation, thus providing proof-of-concept of the therapeutic potential for personalized therapeutic strategies. However, most of the patients will progress within 2 years of starting those therapies. Further, currently there are no targeted therapies that have been shown to be effective in patients with a wild-type BRAF. Thus, there are unmet clinical needs to develop treatments that prevent or overcome resistance to existing therapies for patients with a BRAFV600E/K mutation, and that are effective in patients without a BRAF mutation. In order to facilitate the development of new therapeutic strategies, over the last 5 years we have led a major effort to develop a broad collection of PDX models to reflect the clinical, histological, and genetic heterogeneity of this disease. Our collection of PDX models represents one of the largest collections for any human malignancy, and our initial testing demonstrates that the collection accurately recapitulates the oncogenic drivers and molecular heterogeneity that is observed in patients. This collection also includes a subset of PDX established from patients with acquired resistance to targeted therapies that have been maintained on those agents in vivo to sustain their resistant phenotype. Together these efforts have generated a robust resource to develop, refine, and prioritize new personalized combinatorial therapies for patients. Thus, we propose to establish a multi-disciplinary and multi-institutional PDTC Program focused on the use and continued expansion of our robust melanoma PDX collection to identify new therapeutic approaches that fill important clinical gaps in this disease.                                                                       |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Exome Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                                                               |                      |            |             | RNA Sequencing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 46                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| 16 | TCGA WGS Variants Across 18 Cancer Types                                                                                                      |                      | Controlled | phs003155   | Even though whole genome sequence (WGS) data has been generated and published in many studies, much of this information is not yet processed for use in down-stream analysis. This study's aim is to provide 1342 WGS normal-tumor paired single nucleotide variations (SNP) over 18 different cancer types provided by The Cancer Genome Atlas (TCGA) project. Individual level data for TCGA can be accessed by requesting access for phs000178. This data includes variations within self-reported white and African-American populations. Variations that exist within the tumor tissue but are absent in associated normal organ tissue (as compared to the human reference genome) are reported. Data published includes SNP and small insertions and deletions, which were generated through a pipeline including the VarScan2 variant calling software.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 17 | Discovery of Colorectal Cancer Susceptibility Genes in High-Risk Families                                                                     | Utah                 | Controlled | phs001787   | Colorectal cancer remains one of the most common cancers in the US with 146,970 new diagnoses and 49,920 deaths estimated for 2009. Colon cancer is also one of the most familial of cancers. Individuals with a first-degree relative with colon cancer have a 2- to 3-fold increased risk, and those with more than one first-degree relative with colon cancer, or a single first-degree relative affected at age 50 years, have a 3- to 6-fold greater risk than those with no family history. The most prominent high-risk colorectal cancer susceptibility genes, APC, MLH1, MSH2, MSH6, and PTEN, were all discovered more than a decade ago. Currently, mutation screening of these genes, plus a short list of additional genes that are responsible for a very small fraction of colorectal cancer, plays an important role in the clinical management of individuals with a strong family history of the disease or syndromic evidence for the presence of a gene mutation.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | At the other end of the risk spectrum, genome-wide association studies have identified a number of common alleles with very modest effects on colorectal cancer risk; their clinical utility has yet to be established. However, taken together, the known spectrum of genetic effects only explain about one quarter of the overall familial excess of colorectal cancer. It should be emphasized that, at present, the vast majority of individuals seen at familial cancer clinics are counseled on the basis of their family history alone because they do not have mutations in the known susceptibility genes. Accordingly, the long-term objective of this project is to identify the majority of genes responsible for the unexplained component of inherited colorectal cancer risk. Nextgen DNA sequencing is well suited for application to research questions in genetic susceptibility for which linkage analysis is confounded by extensive genetic heterogeneity. Taking advantage of unparalleled familial cancer genetics resources available through the Utah Population Database, candidate genes have been identified by sequencing all of the gene exons in the human genome from a series of colorectal cancer cases with a very strong family history of colorectal cancer not explained by one of the currently known high-risk susceptibility genes.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                                                               |                      |            |             | Study Design:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Study Type:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                                                               |                      |            |             | Case Set                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                                                               |                      |            |             | Total number of consented subjects: 51                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| 18 | GECCO OICR: Molecular Pathological Epidemiology of Colorectal Cancer                                                                          | GECCO OICR           | Controlled | phs002050   | Colorectal cancer, the second most commonly diagnosed cancer, is a biologically heterogeneous disease. To characterize the molecular attributes of colorectal tumors, we conducted targeted sequencing at the Ontario Cancer Research Institution (OICR) of tumor and matched normal DNA samples from 2,542 colorectal cancer cases in the Genetics and Epidemiology of Colorectal Cancer Consortium (GECCO) and the Colon Cancer Family Registry (CCFR). These consortia are based on cohort and population-based case-control studies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 19 | Whole Genome Sequencing to Discover Familial Myeloma Risk Genes                                                                               |                      | Controlled | phs001819   | We identified germlineÊKDM1AÊtruncating mutations in patients with multiple myeloma (MM), and loss of heterozygosity (LOH) in tumors. KDM1A mutation burden is higher in sporadic MM patients than in controls, and mRNA levels are lower in MM compared with normal plasma cells. KDM1A pharmacological inhibition in vitro promotes myeloma cell proliferation, and in mice promotes plasma cell expansion, enhanced secondary immune response to T cell dependent antigens, and upregulation of MYC oncogene transcriptional targets. Our findings provide important new insights into the role of KDM1A to suppress B cell malignancies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |

From this list, HTAN looks interesting, let's take a look at some numbers to see if there's enough data there to support further analysis

In [27]:
info_query= """
{
  studyDetail(phs_accession:"phs001287"){
    data_types
    numberOfFiles
    numberOfSubjects
    numberOfSamples
  }
}
"""

In [28]:
info_results = runGraphQLQuery(cds_graphql_url, info_query)

In [29]:
print(info_results)

{'data': {'studyDetail': {'data_types': 'Genomic', 'numberOfFiles': 1243, 'numberOfSubjects': 1069, 'numberOfSamples': 1114}}}
